In [23]:
from mapminer.miner import ESRILULCMiner, DEMMiner
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray
import hvplot.xarray
from shapely import Polygon, Point, box

In [39]:
import ee
import os
import json
import xarray as xr
from shapely.geometry import Polygon, Point
from cryptography.fernet import Fernet


class CDLMiner:
    """
    CDLMiner class for extracting data using Google Earth Engine (GEE) and loading it as an xarray DataArray.
    """

    def __init__(self, json_path: str = None):
        """
        Initializes CDLMiner by authenticating using the provided JSON key file.
        
        Args:
            json_path (str): Path to the service account JSON file.
        """
        self.authenticate(json_path)
    
    def fetch(self, lat=None, lon=None, radius=None, polygon=None, daterange="2024-01-01/2024-01-10"):
        """
        Fetches data for the given polygon or point within the specified date range.
        
        Args:
            lat (float): Latitude of the center point (if no polygon provided).
            lon (float): Longitude of the center point (if no polygon provided).
            radius (float): Radius around the point (in meters) to define the area.
            polygon (shapely.geometry.Polygon): Input polygon in EPSG:4326 (if provided).
            daterange (str): Date range to filter data (e.g., "2024-01-01/2024-01-10").
        
        Returns:
            xarray.Dataset: Data as an xarray Dataset.
        """
        # Split the date range into start and end dates
        start_date, end_date = daterange.split("/")

        if polygon is None:
            # Create a buffer polygon around the point
            polygon = Point(lon, lat).buffer(radius / 111000.0)  # Radius in degrees

        # Convert the Shapely polygon to an Earth Engine geometry
        geometry = ee.Geometry.Polygon(list(polygon.exterior.coords))

        # Load the CDL data from Google Earth Engine
        ic = ee.ImageCollection('USDA/NASS/CDL').filterDate(start_date, end_date)

        # Use xarray to open the dataset with the correct engine and projection
        ds = xr.open_dataset(
            ic,
            engine='ee',
            projection=ic.first().select(0).projection(),
            geometry=geometry
        )
        
        return ds

    def authenticate(self, json_path: str = None):
        """
        Authenticates with Google Earth Engine using a service account.
        
        Args:
            json_path (str): Path to the service account JSON file. If not provided,
                            the method will use the encrypted key from the external 'keys' folder.
        """
        # Get the absolute path of the 'keys' folder, assuming it's at the root level of the project
        keys_dir = "../mapminer/keys/"#os.path.abspath(os.path.join(os.path.dirname(__file__), '.', 'keys'))

        if json_path is None:
            # Access secret_key.key and google_service_account.json from the separate 'keys' folder
            secret_key_path = os.path.join(keys_dir, 'secret_key.key')
            service_account_json_path = os.path.join(keys_dir, 'google_service_account.json')

            key = open(secret_key_path, 'rb').read()
            cipher = Fernet(key)

            # Decrypt the Google service account JSON file
            encrypted_json = open(service_account_json_path, 'rb').read()
            decrypted_key = cipher.decrypt(encrypted_json)
            service_account_config = json.loads(decrypted_key)
        else:
            # Load service account configuration from the provided JSON path
            with open(json_path, 'r') as f:
                service_account_config = json.load(f)

        # Authenticate with GEE using service account credentials
        credentials = ee.ServiceAccountCredentials(
            service_account_config['client_email'],
            key_data=json.dumps(service_account_config)
        )
        ee.Initialize(credentials)


if __name__ == '__main__':
    # Initialize CDLMiner with the service account JSON file
    miner = CDLMiner()
    
    # Fetch CDL data for a small area around a given point with a date range
    ds = miner.fetch(lon=-95.665, lat=39.8283, radius=10000, daterange="2020-01-01/2020-08-10")
    
    # Output the results
    print(ds)


<xarray.Dataset> Size: 4MB
Dimensions:     (time: 1, X: 507, Y: 675)
Coordinates:
  * time        (time) datetime64[ns] 8B 2020-01-01
  * X           (X) float32 2kB 2.082e+04 2.085e+04 ... 3.597e+04 3.6e+04
  * Y           (Y) float32 3kB 1.878e+06 1.878e+06 ... 1.858e+06 1.858e+06
Data variables:
    cropland    (time, X, Y) int32 1MB ...
    cultivated  (time, X, Y) int32 1MB ...
    confidence  (time, X, Y) int32 1MB ...
Attributes: (12/20)
    date_range:               [852076800000, 1609459200000]
    description:              <p>The Cropland Data Layer (CDL) is a crop-spec...
    keywords:                 ['cdl', 'cropland', 'landcover', 'nass', 'usda']
    period:                   365
    period_mapping:           ['1997-01-01T00:00:00Z', '1998-01-01T00:00:00Z'...
    product_tags:             ['cdl', 'cropland', 'landcover']
    ...                       ...
    visualization_0_bands:    cropland
    visualization_0_max:      95.0
    visualization_0_min:      11.0
    visual

In [40]:
ds

<xarray.Dataset> Size: 4MB
Dimensions:     (time: 1, X: 507, Y: 675)
Coordinates:
  * time        (time) datetime64[ns] 8B 2020-01-01
  * X           (X) float32 2kB 2.082e+04 2.085e+04 ... 3.597e+04 3.6e+04
  * Y           (Y) float32 3kB 1.878e+06 1.878e+06 ... 1.858e+06 1.858e+06
Data variables:
    cropland    (time, X, Y) int32 1MB ...
    cultivated  (time, X, Y) int32 1MB ...
    confidence  (time, X, Y) int32 1MB ...
Attributes: (12/20)
    date_range:               [852076800000, 1609459200000]
    description:              <p>The Cropland Data Layer (CDL) is a crop-spec...
    keywords:                 ['cdl', 'cropland', 'landcover', 'nass', 'usda']
    period:                   365
    period_mapping:           ['1997-01-01T00:00:00Z', '1998-01-01T00:00:00Z'...
    product_tags:             ['cdl', 'cropland', 'landcover']
    ...                       ...
    visualization_0_bands:    cropland
    visualization_0_max:      95.0
    visualization_0_min:      11.0
    visualization_0_name:     Crop Landcover
    visualization_0_palette:  ffd300,ff2626,00a8e2,ff9e0a,267000,ffff00,70a50...
    crs:                      EPSG:5070

In [ ]:
import xee

In [ ]:
cdl_data

In [ ]:
%%time
from mapminer.miner import 
miner = LandsatMiner()

In [21]:
%%time
lat,lon = 29.32519295,74.46762929
radius = 1000  # 1 km radius

ds = miner.fetch(lat,lon,radius)

CPU times: user 49.2 ms, sys: 179 µs, total: 49.4 ms
Wall time: 1.19 s


In [22]:
ds

<xarray.Dataset> Size: 611kB
Dimensions:      (time: 2, y: 68, x: 59)
Coordinates:
  * y            (y) float64 544B 3.245e+06 3.245e+06 ... 3.243e+06 3.243e+06
  * x            (x) float64 472B 4.474e+05 4.475e+05 ... 4.491e+05 4.492e+05
    spatial_ref  int32 4B 32643
  * time         (time) datetime64[ns] 16B 2024-01-02T05:31:19.404399 2024-01...
Data variables: (12/19)
    qa           (time, y, x) float32 32kB dask.array<chunksize=(1, 68, 59), meta=np.ndarray>
    red          (time, y, x) float32 32kB dask.array<chunksize=(1, 68, 59), meta=np.ndarray>
    blue         (time, y, x) float32 32kB dask.array<chunksize=(1, 68, 59), meta=np.ndarray>
    drad         (time, y, x) float32 32kB dask.array<chunksize=(1, 68, 59), meta=np.ndarray>
    emis         (time, y, x) float32 32kB dask.array<chunksize=(1, 68, 59), meta=np.ndarray>
    emsd         (time, y, x) float32 32kB dask.array<chunksize=(1, 68, 59), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) float32 32kB dask.array<chunksize=(1, 68, 59), meta=np.ndarray>
    swir22       (time, y, x) float32 32kB dask.array<chunksize=(1, 68, 59), meta=np.ndarray>
    coastal      (time, y, x) float32 32kB dask.array<chunksize=(1, 68, 59), meta=np.ndarray>
    qa_pixel     (time, y, x) float32 32kB dask.array<chunksize=(1, 68, 59), meta=np.ndarray>
    qa_radsat    (time, y, x) float32 32kB dask.array<chunksize=(1, 68, 59), meta=np.ndarray>
    qa_aerosol   (time, y, x) float32 32kB dask.array<chunksize=(1, 68, 59), meta=np.ndarray>

In [121]:
lat,lon = 29.32519295,74.46762929
radius = 1000  # 1 km radius
resolution = 1.0  # 10 meter resolution

miner = ESRIBaseMapMiner()
dataset = miner.fetch(lat=lat, lon=lon, radius=radius, resolution=resolution)

In [122]:
dataset

<xarray.DataArray (y: 2293, x: 1999, band: 3)> Size: 14MB
array([[[107, 107,  77],
        [107, 107,  77],
        [107, 107,  77],
        ...,
        [ 99, 100,  69],
        [ 91,  99,  69],
        [100, 107,  76]],

       [[116, 115,  84],
        [107, 107,  77],
        [115, 114,  78],
        ...,
        [ 99, 100,  69],
        [107, 108,  82],
        [ 92, 100,  74]],

       [[124, 122,  91],
        [116, 115,  84],
        [116, 115,  84],
        ...,
...
        ...,
        [187, 163, 133],
        [194, 165, 132],
        [187, 163, 133]],

       [[220, 196, 164],
        [213, 190, 161],
        [220, 196, 164],
        ...,
        [194, 165, 132],
        [196, 169, 133],
        [196, 169, 133]],

       [[212, 194, 163],
        [213, 190, 161],
        [212, 188, 156],
        ...,
        [187, 163, 133],
        [187, 163, 133],
        [187, 163, 133]]], dtype=uint8)
Coordinates:
  * y        (y) float64 18kB 3.416e+06 3.416e+06 ... 3.418e+06 3.418e+06
  * x        (x) float64 16kB 8.289e+06 8.289e+06 ... 8.291e+06 8.291e+06
  * band     (band) <U1 12B 'R' 'G' 'B'
Attributes:
    metadata:  {'date': '20231006'}

In [123]:
dataset.hvplot.rgb(x='x',y='y',height=800,width=800,bands='band',rasterize=True)

:DynamicMap   []
   :RGB   [x,y]   (R,G,B)

In [103]:
dataset["basemap"].crs

AttributeError: 'DataArray' object has no attribute 'crs'

In [62]:
dataset['basemap']['band'] = np.array([0,1,2])
dataset['basemap'].hvplot.rgb(height=600,width=600,bands='band')

Invoked as dynamic_operation(3166193.028983842)
[2024-10-23 19:15:55,488] [ WARNING] parameterized.py:2533 - Callable raised "ValueError('Data must be 3D array to be converted to RGB.')".
Invoked as dynamic_operation(3166193.028983842)


ValueError: Data must be 3D array to be converted to RGB.

:DynamicMap   [y]

In [32]:
500*(1/111/1000)

0.0045045045045045045

In [33]:
500/111/1000

0.0045045045045045045

In [42]:
111*1000

111000